In [17]:
pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Compiling surprise/similarities.pyx because it changed.
      Compiling surprise/prediction_algorithms/matrix_factorization.pyx because it changed.
      Compiling surprise/prediction_algorithms/optimize_baselines.pyx because it changed.
      Compiling surprise/prediction_algorithms/slope_one.pyx because it changed.
      Compiling surprise/prediction_algorithms/co_clustering.pyx because it changed.
      [1/5] Cythonizing surprise/prediction_algorithms/co_clustering.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
              self.avg_cltr_i = avg_cltr_i
              self.avg_cocltr = avg_cocltr
      
              return self
      
          def compute_averages(self, np.ndarray[np.int_t] cltr_u,
                                                   ^
  

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Surprise (Collaborative Filtering)
# If you get an import error, run: !pip install scikit-surprise
from surprise import Dataset, Reader, SVD, KNNWithMeans, accuracy
from surprise.model_selection import train_test_split as surprise_split
from surprise.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'surprise'

In [ ]:
#loading the datasets

ratings = pd.read_csv("data/ratings.csv")
movies = pd.read_csv("data/movies.csv")

In [2]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


No missing values were found for both the two datasets

In [6]:
ratings = ratings.drop(columns=["timestamp"])

Timestamp was not required for rating-based collaborative filtering in this project.

In [7]:
#Filter sparse users and movies
user_counts = ratings["userId"].value_counts()
ratings = ratings[ratings["userId"].isin(user_counts[user_counts >= 5].index)] #in here i kept only users with at least 5 ratings

movie_counts = ratings["movieId"].value_counts()
ratings = ratings[ratings["movieId"].isin(movie_counts[movie_counts >= 5].index)] #at least 5 ratings per movie

In [8]:
#Confirm the join key exists and matches
ratings["movieId"].dtype
movies["movieId"].dtype

dtype('int64')

In [9]:
#both datasets have the same dtype, int64, for the join key, hence they are compatible for merging

In [10]:
#merge
df = ratings.merge(
    movies,
    on="movieId",
    how="left"
)

In [11]:
df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90274 entries, 0 to 90273
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   90274 non-null  int64  
 1   movieId  90274 non-null  int64  
 2   rating   90274 non-null  float64
 3   title    90274 non-null  object 
 4   genres   90274 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 3.4+ MB


In [13]:
df.describe()

,userId,movieId,rating
count,90274.000000,90274.000000,90274.000000
mean,321.462104,16284.192879,3.537358
std,181.825689,31375.602906,1.029858
min,1.000000,1.000000,0.500000
25%,171.000000,1084.000000,3.000000
50%,318.000000,2692.000000,3.500000
75%,474.000000,6711.000000,4.000000
max,610.000000,187595.000000,5.000000


In [14]:
print("ratings shape:", ratings.shape)
print("movies shape:", movies.shape)
print("users:", ratings["userId"].nunique())
print("movies rated:", ratings["movieId"].nunique())
print("rating min/max:", ratings["rating"].min(), ratings["rating"].max())

ratings shape: (90274, 3)
movies shape: (9742, 3)
users: 610
movies rated: 3650
rating min/max: 0.5 5.0
